In [1]:
import os
os.chdir('/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks')
!pwd

/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks


In [2]:
import sys
import os
dev_mode = True
preset = 'mor_python'

if dev_mode:
    try:
        install_prefix = os.environ['CMAKE_INSTALL_PREFIX']
        print(f'Using CMAKE_INSTALL_PREFIX={install_prefix}')
    except KeyError:
        install_prefix = f'../../build/{preset}/install'
    print(f'Using install_prefix={install_prefix}')
    print(f'Adding {install_prefix}/lib/python3/dist-packages to sys.path')
    if os.path.exists(f'{install_prefix}/lib/python3/dist-packages'):
        
        if os.path.abspath(install_prefix) == '/usr':
            sys.path.append(os.path.abspath(
                f'{install_prefix}/lib/python3/dist-packages'))
        else:
            print(f'Inserting {install_prefix}/lib/python3/dist-packages to sys.path')
            sys.path.insert(0, os.path.abspath(
                f'{install_prefix}/lib/python3/dist-packages'))
    print(f"sys.path: {sys.path}")


Using install_prefix=../../build/mor_python/install
Adding ../../build/mor_python/install/lib/python3/dist-packages to sys.path
Inserting ../../build/mor_python/install/lib/python3/dist-packages to sys.path
sys.path: ['/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/build/mor_python/install/lib/python3/dist-packages', '/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/build/python/install/lib/python3/dist-packages', '/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/u2/prudhomm/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/petsc/lib/python3/dist-packages', '/usr/lib/mshr/python3/dist-packages', '/usr/lib/slepc/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
!pwd

/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks


In [4]:
import feelpp
import numpy as np
print(f"feelpp loaded from {feelpp.__file__}")

from feelpp.timing import tic,toc
from feelpp.mor import *


feelpp loaded from /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/build/mor_python/install/lib/python3/dist-packages/feelpp/__init__.py


In [5]:
sys.argv = ['rb']
e = feelpp.Environment(sys.argv, makeCRBOptions(), config=feelpp.localRepository(".",data={'append_np':False}) )

[ Starting Feel++ ] application rb version 0.1 date 2023-Jul-26
 . rb files are stored in /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/./np_1
 .. logfiles :/nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/./np_1/logs


In [6]:
import json
with open('../ad/mormodels.json') as f:
    models = json.load(f)
print(f"models: {models}")
e.setConfigFile('../ad/ad.cfg')

models: {'mormodels': [{'name': 'ad', 'dbroot': '$cfgdir/', 'attribute': 'last_modified', 'load': 'rb', 'output': 'wendland-1'}, {'name': 'ad', 'dbroot': '$cfgdir/', 'attribute': 'last_modified', 'load': 'rb', 'output': 'wendland-2'}, {'name': 'ad', 'dbroot': '$cfgdir/', 'attribute': 'last_modified', 'load': 'rb', 'output': 'wendland-3'}]}
Reading /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/ad.cfg...


In [7]:
ms=feelpp.mor.MORModels(models)
ms.load()



[CRBModelDB::idFromDBLast] model_name=ad
dbbasedir= /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-1
dbbasedir= /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-1 found
looking for dbfilename= /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-1/6d5e9ceb-13ff-4c49-b05f-8ff2c4a9f9e6/ad.crb.json
Last modified db uid: 6d5e9ceb-13ff-4c49-b05f-8ff2c4a9f9e6
model_name=ad
-- crbmodelDB::dbRepository() = /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-1/6d5e9ceb-13ff-4c49-b05f-8ff2c4a9f9e6
[CRBModelDB::idFromDBLast] model_name=notebooks
[CRBModelDB::idFromDBLast] model_name=ad
dbbasedir= /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-2
dbbasedir= /nvme0/prudhomm/Devel/feelpp.quickfix/feelpp/python/notebooks/feelppdb/np_1/../ad/crbdb/wendland-2 found
looki

In [10]:
muspace = ms.parameterSpace()
sampling = muspace.sampling()
sampling.sample(4, "log-random")
print(f"sampling: {sampling[0]}")
j = dict()
j = {"N": 30}
r=ms.run(sampling,j)


sampling: [3.71e-02]


In [11]:
import plotly.graph_objects as go

time = np.linspace(start=0, stop=2.5, num=len(r[0][0].outputs()))

for i in range(len(r)):
    # Create a Plotly figure with a scatter plot
    fig = go.Figure()
    
    for o in range(len(r[i])):
        v=np.array(r[i][o].outputs())
        err=np.array(r[i][o].errors())

        min_values = v*(1-10/100)
        max_values = v*(1+10/100)
        # print(f"v: {v}")
        # print(f"min_values: {min_values}")
        # print(f"max_values: {max_values}")
        
        assert time.size == min_values.size == max_values.size == v.size

        fig.add_trace(go.Scatter(
            x=time,
            y=v,
            mode='lines+markers',
            name=f'wendland-{o}'
        ))
        # fig.add_trace(go.Scatter(x=time + time[::-1],
        #                          y=min_values + max_values[::-1], 
        #                          fill='tozerox',  # Fill the area between the lines
        #                          mode='none',
        #                          showlegend=False,
        #                          line=dict(color='rgba(0,0,0,0)'),  # Invisible line
        #                          name='Region' ) )
    # Update the layout of the figure
    fig.update_layout(
        title=f'wendland outputs for mu={sampling[i]}',
        xaxis_title='Time',
        yaxis_title='Value',
        
        # Use 'category' type for the x-axis to treat it as discrete dates
        xaxis=dict(tickformat='.2f'),
    )
    
    # Show the plot
    fig.show()
    